# Efficient Net - Repurposing/Finetuning
## Introduction

This notebook is an attempt to repurpose and finetune an EfficientNet model to the task of American Sign Language detection for the DSPRO2 project at HSLU.

## Setup
In this section all the necessary libraries are imported.

In [1]:
%pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wandb
import torch
import torch.nn as nn
import torchvision.models as visionmodels
import torchvision.transforms.v2 as transforms
import lightning as L

from lightning.pytorch.loggers import WandbLogger

import nbformat

from typing import Callable

import os

# Our own modules
import models.sweep_helper as sweep_helper

from datapipeline.asl_image_data_module import ASLImageDataModule
from datapipeline.asl_kaggle_image_data_module import ASLKaggleImageDataModule, DEFAULT_TRANSFORMS
from datapipeline.asl_transforms import ExtractHand, RandomBackgroundNoise, RandomRealLifeBackground
from models.asl_model import ASLModel
from models.training import sweep, train_model

INFO:fiftyone.zoo.datasets:Downloading split 'train' to 'C:\Users\kybur\fiftyone\open-images-v7\train' if necessary


Necessary images already downloaded


INFO:fiftyone.utils.openimages:Necessary images already downloaded


Existing download of split 'train' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'train' is sufficient


...

INFO:fiftyone.utils.openimages:Necessary images already downloaded


Existing download of split 'test' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'test' is sufficient


Loading existing dataset 'open-images-v7-test-1000'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'open-images-v7-test-1000'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
c:\Users\kybur\Repos\HSLU\dspro2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ["WANDB_NOTEBOOK_NAME"] = "./dspro2/efficientnet.ipynb"

## Preprocessing
No general data preprocessing is necessary, however there will be random transforms applied to the images during training. The images are resized to 224x224 pixels, which is the input size of the EfficientNet model. The images are also normalized using the mean and standard deviation of the ImageNet dataset, which is the dataset on which the EfficientNet model was pretrained.

The following cells will show the loading of the dataset and the preparation of the mentioned transforms.

In [4]:
PATH = "/exchange/dspro2/silent-speech/ASL_Pictures_Dataset"
PATH = r"C:\Temp\silent-speech"

In [5]:
# datamodule = ASLImageDataModule(path=PATH, val_split_folder="Validation", batch_size=32, num_workers=128)
datamodule = ASLKaggleImageDataModule(path=PATH, train_transforms=DEFAULT_TRANSFORMS.TRAIN, valid_transforms=DEFAULT_TRANSFORMS.VALID, test_transforms=DEFAULT_TRANSFORMS.TEST, batch_size=32, num_workers=20)

## Models

In [6]:
NUM_CLASSES = 28

In [7]:
class ASLEfficientNet(nn.Module):
    def __init__(self, efficientnet_model: visionmodels.efficientnet.EfficientNet, dropout: float = 0.2, unfreeze_features: int = 1, num_classes: int = NUM_CLASSES):
        super().__init__()
        self.model = efficientnet_model
        self.model.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.model.classifier[1].in_features, num_classes)
        )

        unfreeze_features = min(unfreeze_features, len(self.model.features))
        self.unfreeze_features = unfreeze_features
        self.model.features[:len(self.model.features)-unfreeze_features].requires_grad_(False)

    def forward(self, x):
        return self.model(x)

    def get_main_params(self):
        yield from self.model.classifier.parameters()

    def get_finetune_params(self):
        yield from self.model.features.parameters()

## Training

In [8]:
EFFICIENTNET_MODEL = "efficientnet_model"
DROPOUT = "dropout"

In [9]:
def get_pretrained_efficientnet_model(model_type: str):
    if model_type == "b0":
        efficientnet_model = visionmodels.efficientnet_b0(weights=visionmodels.EfficientNet_B0_Weights.DEFAULT)
    elif model_type == "b1":
        efficientnet_model = visionmodels.efficientnet_b1(weights=visionmodels.EfficientNet_B1_Weights.DEFAULT)
    elif model_type == "b2":
        efficientnet_model = visionmodels.efficientnet_b2(weights=visionmodels.EfficientNet_B2_Weights.DEFAULT)
    elif model_type == "b3":
        efficientnet_model = visionmodels.efficientnet_b3(weights=visionmodels.EfficientNet_B3_Weights.DEFAULT)
    elif model_type == "b4":
        efficientnet_model = visionmodels.efficientnet_b4(weights=visionmodels.EfficientNet_B4_Weights.DEFAULT)
    elif model_type == "b5":
        efficientnet_model = visionmodels.efficientnet_b5(weights=visionmodels.EfficientNet_B5_Weights.DEFAULT)
    elif model_type == "b6":
        efficientnet_model = visionmodels.efficientnet_b6(weights=visionmodels.EfficientNet_B6_Weights.DEFAULT)
    elif model_type == "b7":
        efficientnet_model = visionmodels.efficientnet_b7(weights=visionmodels.EfficientNet_B7_Weights.DEFAULT)
    elif model_type == "v2-s":
        efficientnet_model = visionmodels.efficientnet_v2_s(weights=visionmodels.EfficientNet_V2_S_Weights.DEFAULT)
    elif model_type == "v2-m":
        efficientnet_model = visionmodels.efficientnet_v2_m(weights=visionmodels.EfficientNet_V2_M_Weights.DEFAULT)
    elif model_type == "v2-l":
        efficientnet_model = visionmodels.efficientnet_v2_l(weights=visionmodels.EfficientNet_V2_L_Weights.DEFAULT)

    return efficientnet_model

In [10]:
UNFREEZE_FEATURES = "unfreeze_features"


def get_asl_efficientnet_model(efficientnet_model: visionmodels.efficientnet.EfficientNet, dropout: float, unfreeze_features: int = 0) -> nn.Module:
    return ASLEfficientNet(efficientnet_model, dropout=dropout, unfreeze_features=unfreeze_features)

In [11]:
def get_efficientnet_model_from_config(config: dict) -> nn.Module:
    efficientnet_model = get_pretrained_efficientnet_model(config[EFFICIENTNET_MODEL])
    model = get_asl_efficientnet_model(efficientnet_model, config[DROPOUT], config[UNFREEZE_FEATURES])
    return model

In [12]:
run_id = 0
SEED = 42


def train_efficient_net():
    train_model("efficientnet", get_efficientnet_model_from_config, datamodule, get_optimizer=sweep_helper.get_optimizer_with_finetune_group, seed=SEED)

In [13]:
sweep_config = {
    "name": "EfficientNet-V2-S",
    "method": "bayes",
    "metric": {
        "name": f"{ASLModel.VALID_ACCURACY}",
        "goal": "maximize"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 5
    },
    "parameters": {
        EFFICIENTNET_MODEL: {
            "value": "v2-s"
        },
        UNFREEZE_FEATURES: {
           "value": 1
        },
        DROPOUT: {
            "min": 0.1,
            "max": 0.5
        },
        sweep_helper.OPTIMIZER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "value": sweep_helper.OptimizerType.RMSPROP
                },
                sweep_helper.LEARNING_RATE: {
                    "min": 1e-5,
                    "max": 1e-3,
                    "distribution": "log_uniform_values"
                },
                sweep_helper.FINETUNE_LEARNING_RATE: {
                    "min": 1e-7,
                    "max": 1e-5,
                    "distribution": "log_uniform_values"
                },
                sweep_helper.WEIGHT_DECAY: {
                    "min": 0,
                    "max": 1e-3,
                },
                sweep_helper.MOMENTUM: {
                    "min": 0.8,
                    "max": 0.99
                }
            }
        },
        sweep_helper.LEARNING_RATE_SCHEDULER: {
             "parameters": {
                sweep_helper.TYPE: {
                    "values": [sweep_helper.LearningRateSchedulerType.STEP, sweep_helper.LearningRateSchedulerType.EXPONENTIAL]
                },
                sweep_helper.STEP_SIZE: {"value": 5},
                sweep_helper.GAMMA: {
                    "min": 0.1,
                    "max": 0.9
                }
            }
        }
    }
}

In [14]:
sweep(sweep_config=sweep_config, count=20, training_procedure=train_efficient_net)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ./dspro2/efficientnet.ipynb.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: c19qxiku
Sweep URL: https://wandb.ai/dspro2-silent-speech/silent-speech/sweeps/c19qxiku


wandb: Agent Starting Run: 0np1tbma with config:
wandb: 	dropout: 0.29623925801062195
wandb: 	efficientnet_model: v2-s
wandb: 	learning_rate_scheduler: {'gamma': 0.3425453117585129, 'step_size': 5, 'type': 'step'}
wandb: 	optimizer: {'finetune_learning_rate': 1.5212149325822071e-06, 'learning_rate': 0.00021030807148051116, 'momentum': 0.911571459993216, 'type': 'rmsprop', 'weight_decay': 0.0002967036970331395}
wandb: 	unfreeze_features: 1
Seed set to 42
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ./dspro2/efficientnet.ipynb.
wandb: Currently logged in as: v8-luky (dspro2-silent-speech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Split folders already exist, skipping distribution.


...

Metric valid_accuracy improved by 0.006 >= min_delta = 0.0. New best score: 0.590


Epoch 4: 100%|██████████| 3109/3109 [08:56<00:00,  5.79it/s, v_num=86f8]

Metric valid_accuracy improved by 0.005 >= min_delta = 0.0. New best score: 0.595


Epoch 5:  17%|█▋        | 517/3109 [01:10<05:53,  7.34it/s, v_num=86f8] 

epoch,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▄▄▄▄▄▄▄▅▅▅▅▇▇▇▇▇▇▇▇▇███
lr-RMSprop/pg1,███████▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-RMSprop/pg1-momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-RMSprop/pg1-weight_decay,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-RMSprop/pg2,████████▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-RMSprop/pg2-momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-RMSprop/pg2-weight_decay,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▂▅▅▃▄▅▄▅▆▅▇▅▅▆▇▇▅▆▅▅▃▄▄▆█▆▅▅▅▅▅▅▅▄▄▄▅▄▃
train_loss,▇▆█▅▄▄▅▅▄▄▃▄▂▃▃▂▃▃▁▃▃▂▂▂▃▃▂▃▄▃▂▂▂▂▁▅▁▁▃▂
trainer/global_step,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▆▇▇▇▇▇▇▇▇█
valid_accuracy,▁▅▇▇█


In [22]:
finetune_sweep_config_b0 = {
    "name": "EfficientNet-B0-Finetune",
    "method": "bayes",
    "metric": {
        "name": f"{ASLModel.VALID_ACCURACY}",
        "goal": "maximize"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 5
    },
    "parameters": {
        TUNE_TYPE: {
            "value": "finetune"
        },
        EFFICIENTNET_MODEL: {
            "value": "b0"
        },
        UNFREEZE_FEATURES: {
            "min": 1,
            "max": 9
        },
        DROPOUT: {
            "min": 0.1,
            "max": 0.5
        },
        sweep_helper.OPTIMIZER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "value": sweep_helper.OptimizerType.RMSPROP
                },
                sweep_helper.LEARNING_RATE: {
                    "min": 1e-5,
                    "max": 1e-2,
                    "distribution": "log_uniform_values"
                },
                sweep_helper.WEIGHT_DECAY: {
                    "min": 0,
                    "max": 1e-3,
                },
                sweep_helper.MOMENTUM: {
                    "min": 0.8,
                    "max": 0.99
                }
            }
        },
        sweep_helper.LEARNING_RATE_SCHEDULER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "values": [sweep_helper.LearningRateSchedulerType.NONE, sweep_helper.LearningRateSchedulerType.STEP, sweep_helper.LearningRateSchedulerType.EXPONENTIAL]
                },
                sweep_helper.STEP_SIZE: {
                    "min": 1,
                    "max": 10
                },
                sweep_helper.GAMMA: {
                    "min": 0.1,
                    "max": 0.9
                }
            }
        }
    }
}

In [ ]:
sweep(sweep_config=finetune_sweep_config_b0, count=30, training_procedure=train_efficient_net)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ./dspro2/efficientnet.ipynb.


Create sweep with ID: da3s4woi
Sweep URL: https://wandb.ai/dspro2-silent-speech/silent-speech/sweeps/da3s4woi


wandb: Agent Starting Run: 5ya04w7r with config:
wandb: 	dropout: 0.4882254824597369
wandb: 	efficientnet_model: b0
wandb: 	learning_rate_scheduler: {'gamma': 0.12207377267316807, 'step_size': 8, 'type': 'exponential'}
wandb: 	optimizer: {'learning_rate': 0.00014626705462299463, 'momentum': 0.9600000119859484, 'type': 'rmsprop', 'weight_decay': 0.0003786203337267541}
wandb: 	tune_type: finetune
wandb: 	unfreeze_features: 2
Seed set to 42
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ./dspro2/efficientnet.ipynb.
wandb: Currently logged in as: v8-luky (dspro2-silent-speech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A16') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                    | Params | Mode 
-------------------------------------------------------------------
0 | model          | ASLEfficientNetFinetune | 4.0 M  | train
1 | criterion 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Metric train_accuracy improved by 0.089 >= min_delta = 0.0. New best score: 0.759


Validation: |          | 0/? [00:00<?, ?it/s]

Metric valid_accuracy improved by 0.188 >= min_delta = 0.0. New best score: 0.818
Metric train_accuracy improved by 0.049 >= min_delta = 0.0. New best score: 0.808


Validation: |          | 0/? [00:00<?, ?it/s]

Metric train_accuracy improved by 0.055 >= min_delta = 0.0. New best score: 0.862


In [ ]:
finetune_sweep_config_b1 = {
    "name": "EfficientNet-B1-Finetune",
    "method": "bayes",
    "metric": {
        "name": f"{ASLModel.VALID_ACCURACY}",
        "goal": "maximize"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 5
    },
    "parameters": {
        TUNE_TYPE: {
            "value": "finetune"
        },
        EFFICIENTNET_MODEL: {
            "value": "b1"
        },
        UNFREEZE_FEATURES: {
            "min": 1,
            "max": 9
        },
        DROPOUT: {
            "min": 0.1,
            "max": 0.5
        },
        sweep_helper.OPTIMIZER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "value": sweep_helper.OptimizerType.RMSPROP
                },
                sweep_helper.LEARNING_RATE: {
                    "min": 1e-5,
                    "max": 1e-2,
                    "distribution": "log_uniform_values"
                },
                sweep_helper.WEIGHT_DECAY: {
                    "min": 0,
                    "max": 1e-3,
                },
                sweep_helper.MOMENTUM: {
                    "min": 0.8,
                    "max": 0.99
                }
            }
        },
        sweep_helper.LEARNING_RATE_SCHEDULER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "values": [sweep_helper.LearningRateSchedulerType.NONE, sweep_helper.LearningRateSchedulerType.STEP, sweep_helper.LearningRateSchedulerType.EXPONENTIAL]
                },
                sweep_helper.STEP_SIZE: {
                    "min": 1,
                    "max": 10
                },
                sweep_helper.GAMMA: {
                    "min": 0.1,
                    "max": 0.9
                }
            }
        }
    }
}

In [ ]:
sweep(sweep_config=finetune_sweep_config_b1, count=30, training_procedure=train_efficient_net)

## Evaluation

In [15]:
from models.evaluation import Evaluation
from models.training import PROJECT_NAME, ENTITY_NAME

In [36]:
architecture = get_asl_efficientnet_model("finetune", get_pretrained_efficientnet_model("b0"), 0, 1)

evaluation = Evaluation("efficientnet-b0-finetune-8-eval", project=PROJECT_NAME, entity=ENTITY_NAME, model_architecture=architecture, artifact="dspro2-silent-speech/silent-speech/model-9cxm05ng:v", datamodule=datamodule)

In [37]:
model = evaluation.get_model()

CommError: artifact 'model-9cxm05ng:v' not found in 'dspro2-silent-speech/silent-speech'

In [30]:
trainer = L.Trainer(accelerator="auto")

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [46]:
trainer.test(model, datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Split folders already exist, skipping distribution.


c:\Users\kybur\Repos\HSLU\dspro2\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing DataLoader 0: 100%|██████████| 1037/1037 [01:39<00:00, 10.43it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_accuracy_epoch      0.9975572228431702
     test_loss_epoch       0.012213773094117641
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_epoch': 0.012213773094117641,
  'test_accuracy_epoch': 0.9975572228431702}]

In [43]:
import torchvision.datasets as datasets
import lightning as L
from torch.utils.data import Dataset, DataLoader

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet stats
])

class PredictDataModule(L.LightningDataModule):
    def __init__(self, path: str, batch_size: int = 32, num_workers: int = 0):
        super().__init__()
        self.path = path
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage: str):
        self.predict_dataset = datasets.ImageFolder(root=r"C:\Users\kybur\Downloads", transform=data_transforms, allow_empty=True)

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(self.predict_dataset, batch_size=self.batch_size, num_workers=self.num_workers)

In [44]:
predict_datamodule = PredictDataModule(path=PATH, batch_size=32, num_workers=20)
preds = trainer.predict(model, datamodule=predict_datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  8.35it/s]


In [45]:
for pred in preds:
    probabilities = nn.Softmax(dim=-1)(pred)
    pred = torch.argmax(probabilities, dim=-1)
    # print(probabilities)
    print(pred)

tensor([17, 17, 14, 19, 14, 14,  7, 14,  7])


In [40]:
datamodule.test_dataset.classes[7],datamodule.test_dataset.classes[17],datamodule.test_dataset.classes[19]

('H', 'Q', 'S')